# Exploring conformational space of selected macrocycles - "M7" 

In this notebook we present and analyze selected structures, technical notes are [here](www.gitlab.com/user/gosia/icho).

In [19]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
import glob
import py3Dmol

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdMolAlign
from rdkit import rdBase
print(rdBase.rdkitVersion)
import os,time
print( time.asctime())

2016.09.4
Wed Mar 29 16:21:46 2017


In [2]:
# Functions used in this notebook:
def grep_energies_from_sdf_outputs(files):
    energies = []
    for inp in files:
        with open(inp,'r') as f:
            lines = f.readlines()
            for i, line in enumerate(lines):
                if "M  END" in line:
                    energies.append(float(lines[i+1]))
    return energies

### Crystal structure of "M7" macrocycle

In [3]:
cm7 = open('/home/gosia/work/work_on_gitlab/icho/calcs/m7/m7_crystal.xyz','r').read()
vcm7 = py3Dmol.view(width=400,height=400)
vcm7.addModel(cm7,'xyz')
vcm7.setStyle({'stick':{}})
vcm7.setBackgroundColor('0xeeeeee')
vcm7.zoomTo()
vcm7.show()

In [4]:
# find "core" - a part of molecule, which we wish to be most aligned (rmsd-wise) among all the structures
m7 = Chem.AddHs(Chem.MolFromSmiles('N1C(=O)c2nc(C(=O)NCCCNC(=O)c3nc(C(=O)NCCC1)ccc3)ccc2'))
core_m7 = m7.GetSubstructMatch(Chem.MolFromSmiles('C(=O)c1nc(C=O)ccc1'))

In [5]:
templ_m7 = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/m7_crystal.sdf')
m7_crystal = templ_m7[0]

### Conformers generated with the Balloon software:

Conformers were generated in two ways (genetic algorithm):

* Starting with the crystal geometry kept as a template, output: "m7_b_crystal" on the left fig. below

* Starting with the SMILES signature of M7 and allowing to "rebuild the geometry" (option --rebuildGeometry), output: "m7_b_smiles" on the right fig. below

In both cases the Balloon software was asked for 50 conformers (with other parameters set to default values) and it found 10 conformers in the former case and 7 in the latter.

In [8]:
inps_m7_b_sdf = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/results_starting_from_crystalsdf/*.sdf')

In [9]:
inps_m7_b_smi = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/results_starting_from_crystalsmiles/*.sdf')

In [10]:
e_m7_b_sdf = grep_energies_from_sdf_outputs(inps_m7_b_sdf)
e_m7_b_smi = grep_energies_from_sdf_outputs(inps_m7_b_smi)

In [11]:
%%html
<table>
  <tr>
    <td id="m7_b_crystal" ></td>
    <td id="m7_b_smiles"  ></td>
  <tr>
    <td> m7_b_crystal </td>
    <td> m7_b_smiles  </td>  
  </tr>
</table>

,
m7_b_crystal,m7_b_smiles


In [12]:
# write conformers as a list
allmol_m7_b_sdf = []
allmol_m7_b_smi = []
suppl_m7_b_sdf  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/m7_crystal_sdfout.sdf')
suppl_m7_b_smi  = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/balloon/m7_crystal_smilesout.sdf')

for mol in suppl_m7_b_sdf:
    allmol_m7_b_sdf.append(mol)
for mol in suppl_m7_b_smi:
    allmol_m7_b_smi.append(mol) 

In [13]:
# align:
for mol in allmol_m7_b_sdf:
    AllChem.AlignMolConformers(mol,atomIds=core_m7)
for mol in allmol_m7_b_smi:
    AllChem.AlignMolConformers(mol,atomIds=core_m7)  

In [14]:
# view:
p7_b_handles=[]

p7_b_sdf = py3Dmol.view(width=400,height=400)
for mol in allmol_m7_b_sdf:
    mb = Chem.MolToMolBlock(mol)
    p7_b_sdf.addModel(mb,'sdf')
p7_b_sdf.setStyle({'stick':{'radius':'0.15'}})
p7_b_sdf.setBackgroundColor('0xeeeeee')
p7_b_sdf.zoomTo()    
p7_b_handles.append(p7_b_sdf)

p7_b_smi = py3Dmol.view(width=400,height=400)
for mol in allmol_m7_b_smi:
    mb = Chem.MolToMolBlock(mol)
    p7_b_smi.addModel(mb,'sdf')
p7_b_smi.setStyle({'stick':{'radius':'0.15'}})
p7_b_smi.setBackgroundColor('0xeeeeee')
p7_b_smi.zoomTo()    
p7_b_handles.append(p7_b_smi)

In [15]:
p7_b_handles[0].insert('m7_b_crystal')

In [16]:
p7_b_handles[1].insert('m7_b_smiles')

RMSD calculated against the crystal structure is (for the "m7_b_crystal" and the "m7_b_smiles", respectively):

In [20]:
# calculate RMSD against the crystal structure /check that/:
for i, mol in enumerate(allmol_m7_b_sdf):
    name = "m7_b1_"+str(i)
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name, e_m7_b_sdf[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(templ_m7[0]))))    

name = m7_b1_0, E = 48.372890, RMS = 0.593787
name = m7_b1_1, E = 51.783164, RMS = 0.209230
name = m7_b1_2, E = 58.104823, RMS = 2.007561
name = m7_b1_3, E = 55.531843, RMS = 1.840951
name = m7_b1_4, E = 55.688970, RMS = 1.877492
name = m7_b1_5, E = 57.683850, RMS = 2.016376
name = m7_b1_6, E = 56.106820, RMS = 1.799433
name = m7_b1_7, E = 53.693431, RMS = 1.921691
name = m7_b1_8, E = 57.536989, RMS = 1.363563
name = m7_b1_9, E = 52.100671, RMS = 1.380051


In [22]:
# calculate RMSD against the crystal structure /check that/:
for i, mol in enumerate(allmol_m7_b_smi):
    name = "m7_b2_"+str(i)
    print("name = {}, E = {:.6f}, RMS = {:.6f}".format(name, e_m7_b_smi[i], AllChem.GetBestRMS(Chem.RemoveHs(mol),Chem.RemoveHs(templ_m7[0]))))

name = m7_b2_0, E = 54.675946, RMS = 0.844525
name = m7_b2_1, E = 60.504327, RMS = 1.803571
name = m7_b2_2, E = 63.756299, RMS = 1.346777
name = m7_b2_3, E = 64.114107, RMS = 1.917011
name = m7_b2_4, E = 64.617063, RMS = 1.660194
name = m7_b2_5, E = 63.179995, RMS = 1.414668
name = m7_b2_6, E = 62.770633, RMS = 2.053687


All conformers are very much alike (the program finds only *syn-syn* conformers), so we can choose for instance:

* m7_b1_0
* m7_b1_1
* m7_b1_9 (similar E as m1_b1_1, but significantly different RMS)
* m7_b1_7
* m7_b2_0
* m7_b1_4 (m7_b1_3, m7_b1_6 rejected; similar E, but lower RMS)
* m7_b1_5 (rejected: m7_b1_2, m7_b1_8, m7_b2_1)
* m7_b2_6 (rejected: m7_b2_5, m7_b2_2)
* m7_b2_3 (rejected: m7_b2_4)

Below we will align the selected conformers:

In [23]:
selected_b = [allmol_m7_b_sdf[0], allmol_m7_b_sdf[1], allmol_m7_b_sdf[9], 
              allmol_m7_b_sdf[7], allmol_m7_b_smi[0], allmol_m7_b_sdf[4], 
              allmol_m7_b_sdf[5], allmol_m7_b_smi[6], allmol_m7_b_smi[3]]

for mol in selected_b:
    core_mol = mol.GetSubstructMatch(Chem.MolFromSmiles('n1ccccc1'))
    AllChem.AlignMol(mol,m7_crystal,atomMap=list(zip(core_mol,core_m7)))
    
p_b = py3Dmol.view(width=400,height=400)
for mol in selected_b:
    mb = Chem.MolToMolBlock(mol)
    p_b.addModel(mb,'sdf')
p_b.setStyle({'stick':{'radius':'0.15'}})
p_b.setBackgroundColor('0xeeeeee')
p_b.zoomTo()
p_b.show()

### Conformers generated with the RDKit software

RDKit found 6 conformers of similar energy:

In [43]:
# create a list of all structures to be aligned
inps_m7_rdkit = glob.glob('/home/gosia/work/work_on_gitlab/icho/calcs/m7/rdkit/results_crystal_from_smiles/*.sdf')

6

In [44]:
e_m7_rdkit = grep_energies_from_sdf_outputs(inps_m7_rdkit)

[59.65, 64.19, 60.98, 60.56, 60.87, 56.56]

In [45]:
allmol_m7_rdkit = []
suppl_m7_rdkit = Chem.SDMolSupplier('/home/gosia/work/work_on_gitlab/icho/calcs/m7/rdkit/result_smiles.sdf')

for mol in suppl_m7_rdkit:
    allmol_m7_rdkit.append(mol)

This time, instead of visualizing all conformers, we will look directly at their alignment:

In [46]:
# align:
for mol in allmol_m7_rdkit:
    AllChem.AlignMolConformers(mol,atomIds=core_m7)

In [47]:
# view:
p = py3Dmol.view(width=400,height=400)
#p.removeAllModels()
for mol in allmol_m7_rdkit:   
    mb = Chem.MolToMolBlock(mol)
    p.addModel(mb,'sdf')    
p.setStyle({'stick':{'radius':'0.15'}})
p.setBackgroundColor('0xeeeeee')
p.zoomTo()
p.show()

### Summary

todo: align all conformers